In [1]:
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.parallel
import torch.optim as optim
import torch.utils.data
from torch.autograd import Variable

In [2]:
movies = pd.read_csv('/home/nilesh/Desktop/MY FILES/DEEP LEARNING COURSE/7. Boltzmann_Machines/Boltzmann_Machines/ml-1m/movies.dat' , delimiter = '::' , header = None , engine = 'python' , encoding = 'latin-1')

In [3]:
users = pd.read_csv('/home/nilesh/Desktop/MY FILES/DEEP LEARNING COURSE/7. Boltzmann_Machines/Boltzmann_Machines/ml-1m/users.dat' , delimiter = '::' , header = None , engine = 'python' , encoding = 'latin-1')

In [4]:
ratings = pd.read_csv('/home/nilesh/Desktop/MY FILES/DEEP LEARNING COURSE/7. Boltzmann_Machines/Boltzmann_Machines/ml-1m/ratings.dat' , delimiter = '::' , header = None , engine = 'python' , encoding = 'latin-1')

In [5]:
training_set = np.array(pd.read_csv('/home/nilesh/Desktop/MY FILES/DEEP LEARNING COURSE/7. Boltzmann_Machines/Boltzmann_Machines/ml-100k/u1.base' , delimiter='\t') , dtype='int')
testing_set = np.array(pd.read_csv('/home/nilesh/Desktop/MY FILES/DEEP LEARNING COURSE/7. Boltzmann_Machines/Boltzmann_Machines/ml-100k/u1.test' , delimiter='\t') , dtype='int')

In [6]:
total_users = int(max(max(training_set[:,0]),max(testing_set[:,0])))
total_movies = int(max(max(training_set[:,1]),max(testing_set[:,1])))

In [7]:
training_set.shape


(79999, 4)

In [9]:
def convert_data(data):
    ret_data = []
    for i in range(1,total_users+1):
        ratings = np.zeros(total_movies)
        for j in range(0,data.shape[0]):
            if data[j][0] == i:
                ratings[data[j][1] - 1] = data[j][2] 
        ret_data.append(list(ratings))
    return ret_data

In [10]:
len(training_set[0])

4

In [11]:
total_movies

1682

In [12]:
training_set_new = convert_data(training_set)

In [13]:
testing_set_new = convert_data(testing_set)

In [14]:
np.array(testing_set_new).shape

(943, 1682)

In [15]:
xtrain = torch.FloatTensor(training_set_new)
xtest = torch.FloatTensor(testing_set_new)

In [16]:
print(type(xtrain))

<class 'torch.Tensor'>


In [17]:
xtrain[xtrain == 0] = -1
xtrain[xtrain == 1] = 0
xtrain[xtrain == 1] = 0
xtrain[xtrain >= 3] = 1
xtest[xtest == 0] = -1
xtest[xtest ==1 ] = 0
xtest[xtest ==2 ] = 0
xtest[xtest >= 3] = 1

In [32]:
class RBM():
    def __init__(self , nv , nh):
        self.w = torch.randn(nh,nv)
        self.a = torch.randn(1 , nh)
        self.b = torch.randn(1 , nv)
        
    def sample_h(self , x):
        wx = torch.mm(x , self.w.t())
        activation = wx + self.a.expand_as(wx)
        p_h_given_v = torch.sigmoid(activation)
        return p_h_given_v , torch.bernoulli(p_h_given_v)
    
    def sample_v(self , y):
        wy = torch.mm(y , self.w)
        activation = wy + self.b.expand_as(wy)
        p_v_given_h = torch.sigmoid(activation)
        return p_v_given_h , torch.bernoulli(p_v_given_h)
    
    def train(self , v0 , vk , ph0 , phk):
        self.w +=(torch.mm(v0.t() , ph0) - torch.mm(vk.t() , phk)).t()
        self.b += torch.sum((v0 - vk) , 0)
        self.a += torch.sum((ph0 - phk) , 0)

In [36]:
nh = 100
nv = len(xtrain[0])
batch_size = 100
rbm = RBM(nv , nh)

In [42]:
nb_epoch = 10
for epoch in range(1 , nb_epoch + 1):
    train_loss = 0
    ctr = 0.
    for user in range(0 , total_users - batch_size , batch_size):
        vk = xtrain[user:user + batch_size]
        v0 = xtrain[user:user + batch_size]
        ph0,_ = rbm.sample_h(v0)
        for k in range(10):
            _,hk = rbm.sample_h(vk)
            _,vk = rbm.sample_v(hk)
            vk[v0<0] = v0[v0<0]
        phk ,_ = rbm.sample_h(vk)
        rbm.train(v0 , vk , ph0 , phk) 
        train_loss += torch.mean(torch.abs(v0[v0>=0] - vk[v0>=0]))
        ctr += 1.
    print('epoch : ', epoch , 'loss :' , train_loss/ctr)

epoch :  1 loss : tensor(0.2133)
epoch :  2 loss : tensor(0.2229)
epoch :  3 loss : tensor(0.2223)
epoch :  4 loss : tensor(0.2134)
epoch :  5 loss : tensor(0.2189)
epoch :  6 loss : tensor(0.2190)
epoch :  7 loss : tensor(0.2174)
epoch :  8 loss : tensor(0.2154)
epoch :  9 loss : tensor(0.2160)
epoch :  10 loss : tensor(0.2193)


In [43]:
test_loss = 0
ctr = 0.
for user in range(0 , total_users):
    v = xtrain[user:user + 1]
    vt = xtest[user:user + 1]
    if len(vt[vt>=0]) > 0:
        _,h = rbm.sample_h(v)
        _,v = rbm.sample_v(h)
        test_loss += torch.mean(torch.abs(vt[vt>=0] - v[vt>=0]))
        ctr += 1.
print('loss :' , str(test_loss/ctr))

loss : tensor(0.1786)
